In [ ]:
import os
from tqdm import tqdm
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import RMSprop
import random

In [ ]:
# Kategoriler ve veri dizini
KATEGORILER = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]
DIR = "/content/gdrive/MyDrive/Colab Notebooks/garbage_dataset/Garbage_classification/Garbage_classification"
BOYUT = 64

# Veriyi saklamak için liste
veri = []

# Veri yükleme ve işleme
for kategori in KATEGORILER:
    klasor_adresi = os.path.join(DIR, kategori)
    deger = KATEGORILER.index(kategori)
    for resim_adi in tqdm(os.listdir(klasor_adresi)):
        resim_adresi = os.path.join(klasor_adresi, resim_adi)
        resim = cv2.imread(resim_adresi, cv2.IMREAD_COLOR)
        if resim is None:
            print("hata")
        else:
            resim = cv2.resize(resim, (BOYUT, BOYUT))
            veri.append([resim, deger])


In [ ]:
# Veriyi karıştırarak modelin daha iyi öğrenmesini sağlama
random.shuffle(veri)

In [ ]:
# Giriş ve çıkış verileri için listeler
X = []
Y = []

# Veriyi ayrıştırma
for x, y in veri:
    X.append(x)
    Y.append(y)

del veri

In [ ]:
# Veriyi numpy dizilerine dönüştürme
X = np.array(X).reshape(-1, BOYUT, BOYUT, 3)
Y = np.array(Y)


In [ ]:
# Veriyi normalleştirme
X = X / 255.0

In [ ]:
# Veriyi eğitim ve doğrulama kümelerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# Giriş katmanı oluşturma
giris = Input(shape=(64, 64, 3))

In [ ]:
# Evrişimli katmanlar
conv1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(giris)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(64, kernel_size=(3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

# Düzleştirme katmanı
flatten = Flatten()(pool2)

# Tam bağlantı katmanları
dense1 = Dense(128, activation='relu')(flatten)
output = Dense(6, activation='softmax')(dense1)

# Modeli oluşturma
model = Model(inputs=giris, outputs=output)

# Model özetini görüntüleme
model.summary()

# Modeli derleme
model.compile(loss='sparse_categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

# Aşırı öğrenmeyi engellemek için 
model_checkpoint = ModelCheckpoint('model.h5', save_best_only=True, monitor='val_accuracy', mode='max')
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, mode='max')

# Modeli eğitme
history = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping, model_checkpoint])
